## PED 1 - Notebook 2. Ajuste (spin-up) y respuesta a un forzamiento radiativo

En el Notebook 1 hemos entendido el modelo a nivel conceptual: qué ecuación representa, qué términos incluye (ASR, OLR y transporte meridional) y qué tipo de “clima” produce tras integrarlo en el tiempo.

En este Notebook 2 empezamos a **usar el modelo como laboratorio numérico**. El objetivo es trabajar con dos conceptos básicas de los modelos climáticos (sencillos o no):

1. **Spin-up (ajuste del modelo):**  
   antes de analizar resultados hay que integrar el modelo el tiempo suficiente para que alcance un régimen estable (en este caso, un régimen estacional periódico).   Esto plantea una pregunta práctica:     **¿cuántos años hay que integrar?**  En el Notebook 1 usamos “20 años” sin justiicarlo, ahora vamos a comprobar si ese valor es razonable o si es excesivo.

2. **Respuesta a un forzamiento radiativo:**  
   El cambio climático se produce por modificaciones del sistema (principalmente concentraciones de GEI) que modifican
el balance radiativo. Para tener esto en cuenta, el modelo permite introducir un forzamiento radiativo externo sencillo y observar cómo reponde el sistema modificando la temperatura.  


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import climlab


## Parámetros que vas a modificar en este cuaderno

En este notebook vamos a modificar dos cosas:

- **NYEARS_SPINUP**: número de años de integración para que el modelo se ajuste.
- **F**: forzamiento radiativo externo (en W m⁻²), que aplicaremos más adelante.

Empieza probando distintos valores de `NYEARS_SPINUP`. La idea es justificar un valor “suficiente” sin integrar de más.


In [ ]:
# Número de años para el ajuste (spin-up). Cambia este valor.
NYEARS_SPINUP = 20

# Lista de valores alternativos para comparar (puedes modificarla)
SPINUP_TEST_YEARS = [1, 2, 5, 10]


## ¿Cómo decidimos si el spin-up es suficiente?

Para decidir si el modelo está ajustado vamos a observar dos cosas:

1. **Temperatura media global** (promedio sobre latitudes) año a año.  
   Si deja de cambiar apreciablemente, es una señal de que el sistema ya está cerca del régimen estacionario.

2. **Perfil latitudinal de temperatura** en algunos años representativos.  
   Si el perfil apenas cambia entre años sucesivos, la estructura espacial también se ha estabilizado.

La pregunta que debes responder al final de este bloque es:

**¿Qué valor mínimo de años de spin-up consideras razonable para este modelo y por qué?**


In [ ]:
def run_spinup_diagnostics(nyears, Ts_init=None):
    m = climlab.EBM_seasonal()
    lat = m.state['Ts'].domain.axes['lat'].points
    
    # Condición inicial uniforme (si se especifica)
    if Ts_init is not None:
        m.state['Ts'][:] = Ts_init
    years = np.arange(nyears + 1)
    Tmean = np.zeros(nyears + 1)

    Ts0 = np.array(m.state['Ts']).ravel()
    Tmean[0] = float(np.mean(Ts0))

    for y in range(1, nyears + 1):
        m.integrate_years(1)
        Tsy = np.array(m.state['Ts']).ravel()
        Tmean[y] = float(np.mean(Tsy))

    return years, Tmean, lat, m


En la siguiente celda podemos **definimos las condiciones iniciales,** de momento, prueba con el valor 
por defecto, más adelante te pediremos que pruebes las otras opciones sugeridas y obtengas
los mismos gráfico para estos valores. 

In [ ]:
# Condición inicial de temperatura (°C)
# Opciones:
# None        -> condición inicial por defecto de climlab
# valor fijo  -> temperatura uniforme en todas las latitudes (por ejemplo -30 o +30)

Ts_initial = None   # prueba luego con 0.0, -30.0 o +30.0


In [ ]:
results = {}

for ny in SPINUP_TEST_YEARS:
    years, Tmean, lat, m_end = run_spinup_diagnostics(ny, Ts_init=Ts_initial)
    results[ny] = (years, Tmean, np.array(m_end.state['Ts']).ravel())

plt.figure(figsize=(7,4))
for ny in SPINUP_TEST_YEARS:
    years, Tmean, _ = results[ny]
    plt.plot(years, Tmean, marker='o', label=f"{ny} años")

plt.xlabel("Años desde el inicio")
plt.ylabel("Temperatura media global ⟨T⟩ (°C)")
plt.title("Spin-up: evolución de la temperatura media global")
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(7,4))
for ny in SPINUP_TEST_YEARS:
    _, _, Ts_end = results[ny]
    plt.plot(lat, Ts_end, label=f"{ny} años")

plt.xlabel("Latitud (°)")
plt.ylabel("Temperatura superficial Ts (°C)")
plt.title("Perfil latitudinal tras distintos tiempos de spin-up")
plt.grid(True)
plt.legend()
plt.show()


## Dependencia de la condición inicial

Hasta ahora hemos usado la condición inicial “por defecto” del modelo, que es realista al seguir la variación prevista por latitudes (como veíamos en el Notebook 1).  
Sin embargo, en un modelo climático una condición inicial realista puede ser muy difícil de estimar. 

Para comprobar el efecto de las condiciones iniciales vuelve a la celda anterior y ajusta, como se indica, las condiciones iniciales a valores uniformes (-30.0, 0.0 y 30.0).

Observa:
- si el tiempo de ajuste cambia,
- si el estado final es el mismo,
- si aparecen comportamientos transitorios poco realistas durante los primeros años.

Este tipo de pruebas son habituales en modelización climática y ayudan a decidir cuánto tiempo de spin-up es necesario.


A continuación tienes un código que hace esas comparaciones de forma automatizada para 
las diferentes condiciones iniciales.

In [ ]:
INITIAL_CONDITIONS = [None, 0, -30.0, +30.0]

plt.figure(figsize=(7,4))

for Ts0 in INITIAL_CONDITIONS:
    label = "por defecto" if Ts0 is None else f"Ts inicial = {Ts0} °C"
    years, Tmean, lat, _ = run_spinup_diagnostics(NYEARS_SPINUP, Ts_init=Ts0)
    plt.plot(years, Tmean, marker='o', label=label)

plt.xlabel("Años desde el inicio")
plt.ylabel("Temperatura media global ⟨T⟩ (°C)")
plt.title("Spin-up: efecto de la condición inicial")
plt.grid(True)
plt.legend()
plt.show()


### Preguntas para el informe de la PED 1

Con toda esta información para este modelo tan sencillo, responde a estas cuestiones:

1. ¿Cuánto tiempo tarda el modelo en alejarse del patrón de la condición inicial para converger a la solución final?
2. ¿Influye en algo la elección de unas condiciones iniciales realistas en una convergencia rápida? Razona por qué crees que esto será mucho más determinante en un modelo complejo (GCM) que en uno tan sencillo.    


## Respuesta del modelo a un forzamiento radiativo externo

Una vez que el modelo ha alcanzado un régimen estacional estable (tras el spin-up), podemos utilizarlo para estudiar cómo responde el clima a una perturbación externa.

En este modelo, un **forzamiento radiativo externo** se introduce de forma muy sencilla, modificando el término constante de la radiación infrarroja saliente (OLR):

OLR = A + B · Tₛ   →   OLR = (A − F) + B · Tₛ

donde **F** es el forzamiento radiativo (en W m⁻²).

- F > 0 representa un forzamiento positivo (por ejemplo, aumento de gases de efecto invernadero).
- F < 0 representa un forzamiento negativo.

Este procedimiento no pretende reproducir procesos radiativos reales con detalle, sino **aislar el efecto térmico de una perturbación energética global**.

En este notebook vamos a comprobar cómo cambia la temperatura del modelo cuando variamos F, y si la respuesta es suave y progresiva.


In [ ]:
# Forzamiento radiativo externo (W m^-2)
# Modifica este valor para explorar la respuesta del modelo
F = 4.0


Al modificar el forzamiento radiativo, el modelo deja de estar en equilibrio.

Por tanto, es necesario:
1. aplicar el forzamiento,
2. volver a integrar el modelo durante varios años,
3. analizar el nuevo estado climático alcanzado.

Usaremos el mismo número de años de spin-up que has justificado en el bloque anterior.


In [ ]:
#NYEARS_SPINUP = #RELLENAR CON UN VALOR 

# Construimos el modelo base
m_forced = climlab.EBM_seasonal()

# Aplicamos el forzamiento radiativo modificando A en la OLR
m_forced.subprocess['LW'].A -= F

# Integramos para alcanzar el nuevo régimen estacional
m_forced.integrate_years(NYEARS_SPINUP)


Analizaremos la respuesta del modelo de dos formas:

1. El cambio en la **temperatura media global**.
2. El cambio en el **perfil latitudinal de temperatura**.

Después compararemos estos resultados con el caso sin forzamiento (F = 0).


In [ ]:
# Temperatura media global tras el ajuste
Ts_forced = np.array(m_forced.state['Ts']).ravel()
Tmean_forced = float(np.mean(Ts_forced))

print(f"Forzamiento F = {F:.1f} W m^-2")
print(f"Temperatura media global tras el ajuste: {Tmean_forced:.2f} °C")


In [ ]:
# Caso de referencia sin forzamiento
m_control = climlab.EBM_seasonal()
m_control.integrate_years(NYEARS_SPINUP)

Ts_control = np.array(m_control.state['Ts']).ravel()
Tmean_control = float(np.mean(Ts_control))

print(f"Caso sin forzamiento: {Tmean_control:.2f} °C")
print(f"Cambio de temperatura media: {Tmean_forced - Tmean_control:.2f} °C")


In [ ]:
lat = m_forced.state['Ts'].domain.axes['lat'].points

plt.figure(figsize=(7,4))
plt.plot(lat, Ts_control, label="F = 0 (control)")
plt.plot(lat, Ts_forced, label=f"F = {F:.1f} W m$^{{-2}}$")
plt.xlabel("Latitud (°)")
plt.ylabel("Temperatura superficial Ts (°C)")
plt.title("Perfil latitudinal tras el ajuste")
plt.grid(True)
plt.legend()
plt.show()


Observa la figura anterior y responde:

- ¿El calentamiento es uniforme en todas las latitudes?
- ¿Qué regiones se calientan más?
- ¿El patrón latitudinal cambia o se desplaza aproximadamente de forma paralela?

Estas preguntas no tienen una única respuesta “correcta”, pero deben apoyarse en las figuras obtenidas.


### Tareas (para tu entrega)

3. Repite el experimento para al menos **dos valores distintos de F**:
   - uno positivo,
   - uno negativo.

4. Para cada caso:
   - indica el valor de F utilizado,
   - muestra la figura del perfil latitudinal,
   - calcula el cambio en la temperatura media global respecto al caso sin forzamiento.

5. Describe brevemente:
   - si la respuesta del modelo es progresiva,
   - si observas algún comportamiento abrupto o no lineal.

Guarda las figuras que consideres relevantes e inclúyelas en tu entrega.
